# Predicting

Generate and save Y_pred for both methods


In [1]:
# Set notebook to use only one GPU
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from file_functions import *
from models_bachelors import *
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
from keras_uncertainty.models import StochasticClassifier

dataset = load('all_subject_runs')
loaded_inputs = dataset['inputs']
loaded_targets = dataset['targets']
'''
Loads a dictionary with 2 keys: 'inputs', 'targets'. 
Both keys have ndarray containing inputs and targets
of 9 subjects separated by subject.
'''
lockbox = load('lockbox')

Using TensorFlow backend


Keras Uncertainty will use standalone Keras backend

# To-do:

This code was originally intended to save both methods predictions. But MCDropout had to undergo training again because turns out it was actually MCDropConnect being trained 💀💀💀. So now the code only works when methods has a single element.

In [3]:
predictions = {'mcdropout': {'preds_lst': [], 'lockbox_preds_lst': []},
                'mcdropconnect': {'preds_lst': [], 'lockbox_preds_lst': []}
              }

for method, values in predictions.items():
    wts_directory = f'{method}/weights'
    # For each subject, get Y_preds for the test subject, and also get the 
    # Y_preds for the lockboxed data. Then save
    # Y preds for each subject would be shape (456, 4)
    for subject_id in range(0, 9):
        X_test = loaded_inputs[subject_id]
        Y_test = loaded_targets[subject_id]
        wts_path = checkpoint_path = f'{wts_directory}/test_subject_{subject_id}.ckpt'
        model = create_model(method=method, drop_rates=0.2)
        model.load_weights(wts_path)
        # Get Y_preds for test subject
        if method != 'standard':
            model = StochasticClassifier(model)
            Y_preds = model(X_test, num_samples=50)
            # Get lockboxed Y_preds for test subject
            lockbox_Y_preds = model(lockbox['inputs'][subject_id], num_samples=50)
        else:
            Y_preds = model.predict(X_test)
            # Get lockboxed Y_preds for test subject
            lockbox_Y_preds = model(lockbox['inputs'][subject_id])

        '''
          Must append Y_true for both cases too!
        '''
        values['preds_lst'].append(Y_preds)
        values['lockbox_preds_lst'].append(lockbox_Y_preds)



ResourceExhaustedError: Exception encountered when calling layer 'conv2d' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[576,22,1125,25] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

Call arguments received by layer 'conv2d' (type Conv2D):
  • inputs=tf.Tensor(shape=(576, 22, 1125, 1), dtype=float32)

In [ ]:
for method, values in predictions.items():
    values['preds_lst'] = np.array(np.array(values['preds_lst']))
    values['lockbox_preds_lst'] = np.array(values['lockbox_preds_lst'])

save('mcdropout/predictions_mcdropout', predictions['mcdropout'])
save('mcdropconnect/predictions_mcdropconnect', predictions['mcdropconnect'])
